In [1]:
!pip install thop

In [3]:
import sys
from pathlib import Path

# Add the project root to sys.path dynamically
project_root = Path.cwd().parents[1]  # adjust if notebook is nested differently
sys.path.append(str(project_root / "src"))

import torch

from model import MiniCNN, ConvBlock, FCBlock
from train import trainModel
from data import CustomDataset, load_cifar_10_data, check_data_loading, Loader, class_to_idx
from visualizations import plotFmaps_and_activationHist, plotCurves
from utils import unpickle, loadWeights, readJson, genError, saveHistory, evalModel

In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# base = "/content/drive/MyDrive/project1"
base = "/home/fw7th/regularization-ml"

In [ ]:
"""
# Copy once from Drive
!cp /content/drive/MyDrive/project1/cifar-10-python.tar.gz /content/

# Extract locally
!mkdir /content/dataset/
!tar -xvzf /content/cifar-10-python.tar.gz -C /content/dataset/
"""

In [ ]:
# Defining the layers for the baseline model
base_conv_layers = [
    ConvBlock(3, 64, pool=False),
    ConvBlock(64, 64),
    ConvBlock(64, 128, pool=False),
    ConvBlock(128, 128)
]

base_fc_layers = [
    FCBlock(128, 64),
    torch.nn.Linear(64, 10)
]

baseline_model = MiniCNN(base_conv_layers, base_fc_layers)

In [ ]:
train_data, train_labels, val_data, val_labels, test_data, test_labels = load_cifar_10_data()

In [ ]:
# Just sanity testing
image_array = val_data[0]
image_label = val_labels[0]
image = image_array.reshape(3, 32, 32)
image = Image.fromarray(image.transpose(1, 2, 0)) #HWC
plt.imshow(image)
plt.title(image_label)
plt.show()

In [ ]:
train_loader, val_loader, test_loader = Loader(train_data, train_labels, val_data, val_labels, test_data, test_labels)

In [ ]:
# Train the baseline model
history_base = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": [], "last_epoch": 0, "LR": []}
model_type = "baseline"
base_model = trainModel(baseline_model, history_base, train_loader, val_loader, model_type)
saveHistory(history_base, "baseline")

In [ ]:
plotFmaps_and_activationHist(baseline_model, "baseline", "final_epoch", 1)

In [ ]:
plotFmaps_and_activationHist(baseline_model, "baseline", "final_epoch", 3)

In [ ]:
baseline_data = readJson("baseline")
print(baseline_data.keys())

In [ ]:
train_losses = baseline_data["train_loss"]
val_losses = baseline_data["val_loss"]
train_accs = baseline_data["train_acc"]
val_accs = baseline_data["val_acc"]
plotCurves(train_losses, val_losses, train_accs, val_accs, "baseline")

In [ ]:
evalModel(baseline_model, "baseline", "model_best")